<a href="https://colab.research.google.com/github/hemil19/Predicting-House-Prices-In-Bengaluru/blob/master/Predicting_house_price_in_bengaluru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train=pd.read_csv('/content/Train.csv')
test=pd.read_csv('/content/Test.csv')

In [ ]:
train.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [ ]:
train.shape,test.shape

((13320, 9), (1480, 9))

In [ ]:
train.isnull().sum()

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

In [ ]:
test.isnull().sum()

area_type          0
availability       0
location           0
size               2
society          626
total_sqft         0
bath               7
balcony           69
price           1480
dtype: int64

In [ ]:
combine=train.append(test)

In [ ]:
combine.isnull().sum()

area_type          0
availability       0
location           1
size              18
society         6128
total_sqft         0
bath              80
balcony          678
price           1480
dtype: int64

In [ ]:
combine.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [ ]:
combine.reset_index(inplace=True)

In [ ]:
#import re
#combine['location']=[re.sub(r" ?\([^)]+\)", "",str(i)) for i in combine['location']]

In [ ]:
combine['location'].fillna(combine['location'].mode()[0],inplace=True)

In [ ]:
sqft=[]
for i in combine['total_sqft']:
  if len(i.split('-'))>1:
    numbers=[float(i) for i in i.split('-')]  
    sqft.append(np.mean(numbers))
  elif i[-5:]=='Perch':
    sqft.append(368.01*float(i[:-5]))
  elif i[-9:]=='Sq. Yards':
    sqft.append(9*float(i[:-9]))
  elif i[-5:]=='Acres':
    sqft.append(43560*float(i[:-5]))
  elif i[-5:]=='Cents':
    sqft.append(435.6*float(i[:-5]))
  elif i[-6:]=='Guntha':
    sqft.append(1089*float(i[:-6]))
  elif i[-7:]=='Grounds':
    sqft.append(2400*float(i[:-7]))
  else:
    sqft.append(10.76*float(i.replace('Sq. Meter','')))
  #9

In [ ]:
combine['total_sqft']=sqft

In [ ]:
combine.head()

,index,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,11362.56,2.0,1.0,39.07
1,1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,27976.00,5.0,3.0,120.00
2,2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,15494.40,2.0,3.0,62.00
3,3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,16365.96,3.0,1.0,95.00
4,4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,12912.00,2.0,1.0,51.00


In [ ]:
combine.dtypes

index             int64
area_type        object
availability     object
location         object
size             object
society          object
total_sqft      float64
bath            float64
balcony         float64
price           float64
dtype: object

In [ ]:
missingsize_index=combine['size'].isna()[combine['size'].isna()==True].index

In [ ]:
missingsize_index

Int64Index([  579,  1775,  2264,  2809,  2862,  5333,  6423,  6636,  6719,
             7680,  8306,  8565,  8703, 10634, 11019, 11569, 13471, 14480],
           dtype='int64')

In [ ]:
k=combine.groupby('location').agg({'size':pd.Series.mode}).reset_index()

In [ ]:
for i in missingsize_index:
  try:
    combine.loc[i,'size']=k[k['location']==combine.loc[i,'location']]['size']
  except:
    combine.loc[i,'size']=combine['size'].mode()[0]

In [ ]:
combine.isnull().sum()

index              0
area_type          0
availability       0
location           0
size               0
society         6128
total_sqft         0
bath              80
balcony          678
price           1480
dtype: int64

In [ ]:
h=combine.groupby('area_type').agg({'society':pd.Series.mode})

In [ ]:
mapping={}
for i in range(len(h)):
  mapping[h.index[i]]=h['society'][i]

In [ ]:
missingsociety_index=combine['society'].isna()[combine['society'].isna()==True].index

In [ ]:
missingsociety_index

Int64Index([    2,     4,     8,     9,    10,    13,    19,    20,    23,
               25,
            ...
            14779, 14782, 14783, 14784, 14786, 14787, 14789, 14792, 14793,
            14796],
           dtype='int64', length=6128)

In [ ]:
for i in missingsociety_index:
    combine.loc[i,'society']=mapping[combine.loc[i,'area_type']]

In [ ]:
combine.isnull().sum()

index              0
area_type          0
availability       0
location           0
size               0
society            0
total_sqft         0
bath              80
balcony          678
price           1480
dtype: int64

In [ ]:
combine.dtypes

index             int64
area_type        object
availability     object
location         object
size             object
society          object
total_sqft      float64
bath            float64
balcony         float64
price           float64
dtype: object

In [ ]:
combine.describe()

,index,total_sqft,bath,balcony,price
count,14800.000000,1.480000e+04,14720.000000,14122.000000,13320.000000
mean,6067.500000,1.680538e+04,2.698709,1.582141,112.565627
std,4059.584065,2.232502e+04,1.343932,0.819147,148.971674
min,0.000000,1.076000e+01,1.000000,0.000000,8.000000
25%,2219.750000,1.179296e+04,2.000000,1.000000,50.000000
50%,5919.500000,1.364368e+04,2.000000,2.000000,72.000000
75%,9619.250000,1.791540e+04,3.000000,2.000000,120.000000
max,13319.000000,1.518041e+06,40.000000,3.000000,3600.000000


In [ ]:
combine['bath'].fillna(combine['bath'].median(),inplace=True)
combine['balcony'].fillna(combine['balcony'].median(),inplace=True)

In [ ]:
combine.isnull().sum()

index              0
area_type          0
availability       0
location           0
size               0
society            0
total_sqft         0
bath               0
balcony            0
price           1480
dtype: int64

In [ ]:
combine[combine['size']=='43 Bedroom']

,index,area_type,availability,location,size,society,total_sqft,bath,balcony,price
4684,4684,Plot Area,Ready To Move,Munnekollal,43 Bedroom,Theanmp,25824.0,40.0,0.0,660.0


In [ ]:
combine['size']=combine['size'].replace('43 Bedroom','4 Bedroom')

In [ ]:
combine.head()

,index,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,11362.56,2.0,1.0,39.07
1,1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,27976.00,5.0,3.0,120.00
2,2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,Sryalan,15494.40,2.0,3.0,62.00
3,3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,16365.96,3.0,1.0,95.00
4,4,Super built-up Area,Ready To Move,Kothanur,2 BHK,GrrvaGr,12912.00,2.0,1.0,51.00


In [ ]:
combine['availability'].unique()

array(['19-Dec', 'Ready To Move', '18-May', '18-Feb', '18-Nov', '20-Dec',
       '17-Oct', '21-Dec', '19-Sep', '20-Sep', '18-Mar', '20-Feb',
       '18-Apr', '20-Aug', '18-Oct', '19-Mar', '17-Sep', '18-Dec',
       '17-Aug', '19-Apr', '18-Jun', '22-Dec', '22-Jan', '18-Aug',
       '19-Jan', '17-Jul', '18-Jul', '21-Jun', '20-May', '19-Aug',
       '18-Sep', '17-May', '17-Jun', '21-May', '18-Jan', '20-Mar',
       '17-Dec', '16-Mar', '19-Jun', '22-Jun', '19-Jul', '21-Feb',
       'Immediate Possession', '19-May', '17-Nov', '20-Oct', '20-Jun',
       '19-Feb', '21-Oct', '21-Jan', '17-Mar', '17-Apr', '22-May',
       '19-Oct', '21-Jul', '21-Nov', '21-Mar', '16-Dec', '22-Mar',
       '20-Jan', '21-Sep', '21-Aug', '14-Nov', '19-Nov', '15-Nov',
       '16-Jul', '15-Jun', '17-Feb', '20-Nov', '20-Jul', '16-Sep',
       '15-Oct', '15-Dec', '16-Oct', '22-Nov', '15-Aug', '17-Jan',
       '16-Nov', '20-Apr', '16-Jan', '14-Jul', '16-Apr'], dtype=object)

In [ ]:
val=[]
for i in combine['availability'].unique():
  if i not in ["Ready To Move","Immediate Possession"]:
    val.append(i)

In [ ]:
len(val)

80

In [ ]:
n=pd.to_datetime([i+'-2020' for i in val])-pd.to_datetime('2016-12-08')

In [ ]:
aa=['1472 days', '1257 days', '1167 days', '1441 days',
                '1473 days', '1409 days', '1474 days', '1381 days',
                '1382 days', '1196 days', '1169 days', '1227 days',
                '1351 days', '1410 days', '1197 days', '1379 days',
                '1471 days', '1348 days', '1228 days', '1288 days',
                '1475 days', '1140 days', '1349 days', '1137 days',
                '1317 days', '1318 days', '1291 days', '1259 days',
                '1350 days', '1380 days', '1256 days', '1287 days',
                '1260 days', '1136 days', '1198 days', '1470 days',
                '1194 days', '1289 days', '1292 days', '1319 days',
                '1170 days', '1258 days', '1440 days', '1412 days',
                '1290 days', '1168 days', '1413 days', '1139 days',
                '1195 days', '1226 days', '1261 days', '1411 days',
                '1321 days', '1444 days', '1199 days', '1469 days',
                '1200 days', '1138 days', '1383 days', '1352 days',
                '1437 days', '1442 days', '1438 days', '1316 days',
                '1285 days', '1166 days', '1443 days', '1320 days',
                '1378 days', '1407 days', '1468 days', '1408 days',
                '1445 days', '1346 days', '1135 days', '1439 days',
                '1229 days', '1134 days', '1314 days', '1225 days']

In [ ]:
nodays_dict={}
for i in range(0,len(val)):
  nodays_dict[val[i]]=aa[i][:4]

In [ ]:
nodays_dict['Ready To Move']=0
nodays_dict['Immediate Possession']=1

In [ ]:
nodays_dict

{'14-Jul': '1314',
 '14-Nov': '1437',
 '15-Aug': '1346',
 '15-Dec': '1468',
 '15-Jun': '1285',
 '15-Nov': '1438',
 '15-Oct': '1407',
 '16-Apr': '1225',
 '16-Dec': '1469',
 '16-Jan': '1134',
 '16-Jul': '1316',
 '16-Mar': '1194',
 '16-Nov': '1439',
 '16-Oct': '1408',
 '16-Sep': '1378',
 '17-Apr': '1226',
 '17-Aug': '1348',
 '17-Dec': '1470',
 '17-Feb': '1166',
 '17-Jan': '1135',
 '17-Jul': '1317',
 '17-Jun': '1287',
 '17-Mar': '1195',
 '17-May': '1256',
 '17-Nov': '1440',
 '17-Oct': '1409',
 '17-Sep': '1379',
 '18-Apr': '1227',
 '18-Aug': '1349',
 '18-Dec': '1471',
 '18-Feb': '1167',
 '18-Jan': '1136',
 '18-Jul': '1318',
 '18-Jun': '1288',
 '18-Mar': '1196',
 '18-May': '1257',
 '18-Nov': '1441',
 '18-Oct': '1410',
 '18-Sep': '1380',
 '19-Apr': '1228',
 '19-Aug': '1350',
 '19-Dec': '1472',
 '19-Feb': '1168',
 '19-Jan': '1137',
 '19-Jul': '1319',
 '19-Jun': '1289',
 '19-Mar': '1197',
 '19-May': '1258',
 '19-Nov': '1442',
 '19-Oct': '1411',
 '19-Sep': '1381',
 '20-Apr': '1229',
 '20-Aug': '

In [ ]:
combine['availability']=[int(nodays_dict[i]) for i in combine['availability']]

In [ ]:
combine.head()

,index,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,0,Super built-up Area,1472,Electronic City Phase II,2 BHK,Coomee,11362.56,2.0,1.0,39.07
1,1,Plot Area,0,Chikka Tirupathi,4 Bedroom,Theanmp,27976.00,5.0,3.0,120.00
2,2,Built-up Area,0,Uttarahalli,3 BHK,Sryalan,15494.40,2.0,3.0,62.00
3,3,Super built-up Area,0,Lingadheeranahalli,3 BHK,Soiewre,16365.96,3.0,1.0,95.00
4,4,Super built-up Area,0,Kothanur,2 BHK,GrrvaGr,12912.00,2.0,1.0,51.00


In [ ]:
combine.isnull().sum()

index              0
area_type          0
availability       0
location           0
size               0
society            0
total_sqft         0
bath               0
balcony            0
price           1480
dtype: int64

In [ ]:
combine.describe()

,index,availability,total_sqft,bath,balcony,price
count,14800.000000,14800.000000,1.480000e+04,14800.000000,14800.000000,13320.000000
mean,6067.500000,272.875608,1.680538e+04,2.694932,1.601284,112.565627
std,4059.584065,539.154231,2.232502e+04,1.341274,0.804918,148.971674
min,0.000000,0.000000,1.076000e+01,1.000000,0.000000,8.000000
25%,2219.750000,0.000000,1.179296e+04,2.000000,1.000000,50.000000
50%,5919.500000,0.000000,1.364368e+04,2.000000,2.000000,72.000000
75%,9619.250000,0.000000,1.791540e+04,3.000000,2.000000,120.000000
max,13319.000000,1475.000000,1.518041e+06,40.000000,3.000000,3600.000000


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
combine['total_sqft']=scaler.fit_transform(np.array(combine['total_sqft']).reshape(-1,1))
combine['availability']=scaler.fit_transform(np.array(combine['availability']).reshape(-1,1))

In [ ]:
combine.head()

,index,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,0,Super built-up Area,0.997966,Electronic City Phase II,2 BHK,Coomee,0.007478,2.0,1.0,39.07
1,1,Plot Area,0.000000,Chikka Tirupathi,4 Bedroom,Theanmp,0.018422,5.0,3.0,120.00
2,2,Built-up Area,0.000000,Uttarahalli,3 BHK,Sryalan,0.010200,2.0,3.0,62.00
3,3,Super built-up Area,0.000000,Lingadheeranahalli,3 BHK,Soiewre,0.010774,3.0,1.0,95.00
4,4,Super built-up Area,0.000000,Kothanur,2 BHK,GrrvaGr,0.008499,2.0,1.0,51.00


In [ ]:
X = combine[combine['price'].isnull()!=True].drop(['index','price'], axis=1)
y = combine[combine['price'].isnull()!=True]['price']

test = combine[combine['price'].isnull()==True].drop(['index','price'], axis=1)

X.shape, y.shape, test.shape

((13320, 8), (13320,), (1480, 8))

In [ ]:
X.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony
0,Super built-up Area,0.997966,Electronic City Phase II,2 BHK,Coomee,0.007478,2.0,1.0
1,Plot Area,0.000000,Chikka Tirupathi,4 Bedroom,Theanmp,0.018422,5.0,3.0
2,Built-up Area,0.000000,Uttarahalli,3 BHK,Sryalan,0.010200,2.0,3.0
3,Super built-up Area,0.000000,Lingadheeranahalli,3 BHK,Soiewre,0.010774,3.0,1.0
4,Super built-up Area,0.000000,Kothanur,2 BHK,GrrvaGr,0.008499,2.0,1.0


In [ ]:
!pip install catboost

     |████████████████████████████████| 64.8MB 70kB/s 


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=42)

In [ ]:
from catboost import CatBoostRegressor
m = CatBoostRegressor(n_estimators=5000,
                           random_state=1994,
                           eval_metric='MSLE',
                           learning_rate=0.04, 
                           max_depth=5,
                           cat_features=[0,2,3,4])
m.fit(X_train, y_train,
          eval_set=[(X_train,y_train),(X_test, y_test)],
          early_stopping_rounds=200,
          verbose=200)
y_pred=m.predict(X_test)
print(1 - np.sqrt(np.square(np.log10(y_pred +1) - np.log10(y_test +1)).mean()))

0:	learn: 0.5776454	test: 0.5774680	test1: 0.6103042	best: 0.6103042 (0)	total: 14ms	remaining: 1m 9s
200:	learn: 0.1240104	test: 0.1109039	test1: 0.1189477	best: 0.1189477 (200)	total: 2.3s	remaining: 55s
400:	learn: nan	test: 0.0998534	test1: 0.1086254	best: 0.1085751 (395)	total: 4.5s	remaining: 51.6s
600:	learn: nan	test: nan	test1: 0.1026114	best: 0.1026114 (600)	total: 6.75s	remaining: 49.4s
800:	learn: nan	test: nan	test1: 0.0998088	best: 0.0996280 (776)	total: 9.1s	remaining: 47.7s
1000:	learn: nan	test: nan	test1: 0.0981020	best: 0.0981020 (1000)	total: 11.5s	remaining: 46.1s
1200:	learn: nan	test: nan	test1: 0.0973616	best: 0.0972862 (1152)	total: 13.9s	remaining: 44s
1400:	learn: nan	test: nan	test1: 0.0967990	best: 0.0964814 (1362)	total: 16.2s	remaining: 41.7s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.09648142962
bestIteration = 1362

Shrink model to first 1363 iterations.
0.8651017944775723


In [ ]:
test.shape

(1480, 8)

In [ ]:
err = []
y_pred_tot_cb = []
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
i = 1
for train, testing in kf.split(X):
  x_train, x_val = X.iloc[train], X.iloc[testing]
  y_train, y_val = y[train], y[testing]
  m = CatBoostRegressor(n_estimators=5000,
                           random_state=1994,
                           eval_metric='MSLE',
                           learning_rate=0.04, 
                           max_depth=5,
                           cat_features=[0,2,3,4])
  m.fit(x_train, y_train,
          eval_set=[(x_train,y_train),(x_val, y_val)],
          early_stopping_rounds=200,
          verbose=200)
  pred_y = m.predict(x_val)
  print("err_lgm: ",1 - np.sqrt(np.square(np.log10(pred_y +1) - np.log10(y_val +1)).mean()))
  err.append(1 - np.sqrt(np.square(np.log10(pred_y +1) - np.log10(y_val +1)).mean()))
  pred_test = m.predict(test)
  i = i + 1
  y_pred_tot_cb.append(pred_test)

0:	learn: 0.5817592	test: 0.5815260	test1: 0.5807515	best: 0.5807515 (0)	total: 15.3ms	remaining: 1m 16s
200:	learn: 0.1270218	test: 0.1121129	test1: 0.1291201	best: 0.1291201 (200)	total: 2.25s	remaining: 53.8s
400:	learn: 0.1143815	test: 0.0995656	test1: 0.1166401	best: 0.1166393 (399)	total: 4.35s	remaining: 49.9s
600:	learn: 0.1067791	test: nan	test1: 0.1109036	best: 0.1108743 (599)	total: 6.55s	remaining: 47.9s
800:	learn: 0.1019891	test: nan	test1: 0.1088426	best: 0.1088264 (799)	total: 8.79s	remaining: 46.1s
1000:	learn: 0.0995356	test: nan	test1: 0.1084970	best: 0.1084005 (978)	total: 10.9s	remaining: 43.6s
1200:	learn: 0.0963063	test: nan	test1: 0.1081677	best: 0.1080428 (1166)	total: 13.2s	remaining: 41.8s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.1080428257
bestIteration = 1166

Shrink model to first 1167 iterations.
err_lgm:  0.8572479745727917
0:	learn: 0.5860581	test: 0.5859609	test1: 0.5497068	best: 0.5497068 (0)	total: 14.9ms	remaining: 1m 14s

In [ ]:
err

[0.8572479745727917,
 0.8696595317866835,
 0.8411887632020476,
 0.8551168191385228,
 0.8665490301206116]

In [ ]:
np.mean(y_pred_tot_cb,0)

array([ 73.15201911, 352.66553517, 193.14219374, ...,  47.00199958,
        56.11521241,  56.07831056])

In [ ]:
pd.DataFrame(np.mean(y_pred_tot_cb,0),columns=['price']).to_csv('house_07.csv',index=False)#0.87015